In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from keras import backend as K
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import time

In [7]:
mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

4422102/4422102 [==============================] - 2s 1us/step


In [21]:
class BaseDNN(tf.keras.Model):
    def __init__(self, node_counts, dropout_ratios=None, 
                 activation='relu', optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)):
        super(BaseDNN, self).init__()
        self.node_counts = node_counts
        self.dropout_ratios = [0 for _ in range(len(self.node_counts))] if dropout_ratios is None else dropout_ratios
        
        self.activation = activation
        self.optimizer=optimizer
        self.loss=loss

    def call(self, x):
        for n_nodes, dropout_ratio in zip(self.node_counts, self.dropout_ratios):
            x = layers.Dense(n_nodes, activation=self.activation)(x)
            x = layers.Dropout(dropout_ratio)(x)
        return x


class ConvolutionalLayers(tf.keras.Model):
    def __init__(self, filters, kernel_sizes, activation, strides=None, paddings=None,
                 optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)):
        super(ConvolutionalLayers, self).__init__()
        self.filters = filters
        self.kernels_sizes = kernel_sizes
        self.activation = activation

        if strides is None:
            self.strides = []
            for _ in range(len(filters)):
                self.strides = (1, 1)
        else:
            self.strides = strides

        if paddings is None:
            for _ in range(len(filters)):
                self.filters = "valid"
        else:
            self.paddings = paddings

        self.optimizer = optimizer
        self.loss = loss

    def call(self, x):
        for n_filters, kernel_size, stride, padding in zip(self.filters, self.kernels_sizes, self.strides, self.paddings):
            x = layers.Conv2D(n_filters, kernel_size, strides=stride, padding=padding, activation=self.activation)
        return x

In [22]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(model, data, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(data, training=True)
        loss = model.loss(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(model, data, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(data, training=False)
    t_loss = model.loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

model_input = tf.keras.layers.Input(shape=(64,64,3,), name="input_layer")
x = ConvolutionalLayers()(model_input)
x = BaseDNN()(x)

model = tf.keras.Model(inputs=model_input, outputs=x)

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in zip(X_train, y_train):
        train_step(images, labels)

    for test_images, test_labels in zip(X_test, y_test):
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )


In [ ]:
class BaseDNN(tf.keras.Model):
    def __init__(self, node_counts, dropout_ratios=None, 
                 activation='relu', optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)):
        super(BaseDNN, self).init__()
        self.node_counts = node_counts
        self.dropout_ratios = [0 for _ in range(len(self.node_counts))] if dropout_ratios is None else dropout_ratios
        
        self.activation = activation
        self.optimizer=optimizer
        self.loss=loss

    def call(self, x):
        for n_nodes, dropout_ratio in zip(self.node_counts, self.dropout_ratios):
            x = layers.Dense(n_nodes, activation=self.activation)(x)
            x = layers.Dropout(dropout_ratio)(x)
        return x


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(model, data, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(data, training=True)
        loss = model.loss(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(model, data, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(data, training=False)
    t_loss = model.loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)